In [2]:
%%capture
!pip install youtube-dl

     |████████████████████████████████| 1.9 MB 824 kB/s eta 0:00:01


In [9]:
!youtube-dl -j --flat-playlist "https://www.youtube.com/c/heritageboard/playlists?view=1&sort=dd&shelf_id=0" > pl.json

In [14]:
!cat pl.json | awk -F'"url": "' '{print $2}'|awk -F'"' '{print $1}' | while read i;do youtube-dl -j --flat-playlist $i >> pl_videos.json || echo $i >> retry;done

In [15]:
!youtube-dl -j --flat-playlist "https://www.youtube.com/c/heritageboard/videos?view=0&sort=dd&shelf_id=0" > uploads.json

In [22]:
import json
import requests
cc_by = []
other = []
retry = []
seen = []
lic = '"contents":[{"runs":[{"text":"Creative Commons Attribution licence (reuse allowed)"'
with open("pl_videos.json") as pl_videos:
    for line in pl_videos.readlines():
        line_data = json.loads(line.strip())
        if line_data['id'] in seen:
            continue
        req = requests.get(f"https://www.youtube.com/watch?v={line_data['id']}")
        if req.status_code != 200:
            retry.append(line_data['id'])
        if lic in req.text:
            cc_by.append(line_data['id'])
        else:
            other.append(line_data['id'])
        seen.append(line_data['id'])

In [29]:
with open('proc.json', 'w') as outfile:
    json.dump({'cc-by': cc_by, 'other': other, 'retry': retry}, outfile)